In [9]:
from music21 import converter, instrument, note, chord
notes = []
midi = converter.parse("alb_esp1.mid")
notes_to_parse = None
parts = instrument.partitionByInstrument(midi)
if parts: # file has instrument parts
    notes_to_parse = parts.parts[0].recurse()
    print(parts.parts)
else: # file has notes in a flat structure
    notes_to_parse = midi.flat.notes
for element in notes_to_parse:
    if isinstance(element, note.Note):
        notes.append(str(element.pitch))
    elif isinstance(element, chord.Chord):
        notes.append('.'.join(str(n) for n in element.normalOrder))
notes


<music21.stream.iterator.StreamIterator for Score:0xa1917b240 @:0>


['A5',
 'A3',
 'E6',
 'E4',
 'D6',
 'D4',
 'E6',
 'E4',
 'D6',
 'D4',
 'C6',
 'C4',
 'D6',
 'D4',
 'E6',
 'E4',
 'F6',
 'F4',
 'G6',
 'G4',
 'E6',
 'E4',
 'F6',
 'F4',
 'E6',
 'E4',
 'D6',
 'E6',
 'D4',
 'E4',
 'D6',
 'D4',
 'C6',
 'C4',
 'B-5',
 'B-3',
 'A2',
 'A3',
 'E3',
 'C#4',
 'A3',
 'E4',
 'C#4',
 'A4',
 'E4',
 'C#5',
 'A4',
 'E5',
 'C#5',
 'A4',
 'C#4',
 'C#5',
 'E4',
 'E5',
 'A4',
 'A5',
 'C#5',
 'C#6',
 'E5',
 'E6',
 'A5',
 'C#6',
 'E5',
 'A5',
 'C#5',
 'E5',
 'A4',
 'C#5',
 'E4',
 'A4',
 'C#4',
 'E5',
 'A4',
 'C#5',
 'E4',
 'A4',
 'C#4',
 'E4',
 'A3',
 'C#4',
 'A3',
 'A3',
 'E3',
 'A2',
 'A5',
 'A3',
 'E6',
 'E4',
 'D6',
 'D4',
 'E6',
 'E4',
 'D6',
 'D4',
 'C6',
 'C4',
 'D6',
 'D4',
 'E6',
 'E4',
 'F6',
 'F4',
 'G6',
 'G4',
 'E6',
 'E4',
 'F6',
 'F4',
 'E6',
 'E4',
 'D6',
 'E6',
 'D4',
 'E4',
 'D6',
 'D4',
 'C6',
 'C4',
 'B-5',
 'B-3',
 'A5',
 'A3',
 'B-5',
 'B-3',
 'C6',
 'C4',
 'D6',
 'D4',
 'F6',
 'F4',
 'D6',
 'E6',
 'D4',
 'E4',
 'D6',
 'D4',
 'C6',
 'C4',
 'B-5',
 'B-3